<a href="https://colab.research.google.com/github/jarvisx17/Sentiment-Analysis/blob/main/Japanese_Sentiment_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tune Bert For Japanese Text

In [ ]:
# Install required libraries
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgrad

In [ ]:
!pip install sentencepiece
!pip install fugashi
!pip install unidic_lite
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=c0a61c01040c97b7250f7a07684d0e07679f6b7367ebb9c95dce0fc136a2966a
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic_lite


## Hugging Face Model

#**Model**

# 1. Activate GPU and Install Dependencies

In [ ]:
!pip install datasets

In [ ]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
import pandas as pd
torch.cuda.is_available()

True

#2. Preprocess data

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset_dict = load_dataset("jarvisx17/chABSA")
dataset = dataset_dict['train']

In [ ]:
dataset

Dataset({
    features: ['text', 'target'],
    num_rows: 2813
})

In [ ]:
# dataset = pd.read_csv('/content/drive/MyDrive/Bert/Japanese_sentiment.csv').drop(columns='Id')

In [ ]:
# Split the dataset into training and test sets
small_train_dataset = dataset.select(range(2500))
small_test_dataset = dataset.select(range(2500, len(dataset)))

# Display the first two samples of each split
print(small_train_dataset[:2])
print(small_test_dataset[:2])


{'text': ['損益につきましては、前連結会計年度からの豊富な繰越工事に支えられ、年間を通じて工場稼動は安定的に推移しましたが、指名停止の影響による受注量の大幅な減少により、翌年度以降の製作原価の上昇が見込まれることから、工事損失引当金を計上することとなり、完成工事高137億5千万円(前年同期比31億3千万円減・18.6％減)、営業利益5億6千万円(前年同期比12億9千万円減・69.7％減)となりました', 'この結果、営業利益は１億88百万円増（10.0％増）の20億69百万円となった'], 'target': [0, 1]}
{'text': ['音楽やアーティスト関連の市場動向といたしましては、平成28年における音楽ソフト（オーディオレコード及び音楽ビデオ合計）の生産数量は212,293千枚（前年同期比5.1%減）、販売金額では245,657百万円（前年同期比3.5%減）と減少しております', '当連結会計年度（以下、当期という）におけるわが国経済は、政府や日銀による経済・金融政策を背景に雇用・所得環境の改善が見られたものの、中国をはじめとする新興諸国の成長鈍化や資源安の長期化に加えて、英国のEU（欧州連合）離脱問題や米国新政権移行に伴う影響などによる株式市場や為替相場の急激な変動もあり、景気の先行きは不透明な状況が続きました'], 'target': [0, 0]}


In [ ]:
# from datasets import Dataset

# T_dataset = Dataset.from_pandas(dataset)
# T_dataset.reset_format()
# T_dataset

In [ ]:
T_dataset = dataset.rename_column("target", "labels")

In [ ]:
T_dataset

Dataset({
    features: ['text', 'labels'],
    num_rows: 2813
})

In [ ]:
small_train_dataset = T_dataset.shuffle(seed=42).select([i for i in list(range(2500))])
small_test_dataset = T_dataset.shuffle(seed=42).select([i for i in list(range(312))])

In [ ]:
# Set DistilBERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("abhishek/autonlp-japanese-sentiment-59363")

tokenizer_config.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/236k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7b98ad70f250> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/312 [00:00<?, ? examples/s]

In [ ]:
tokenized_train.set_format("torch")
tokenized_test.set_format("torch")

In [ ]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. Training the model

In [ ]:
# Define DistilBERT as our base model:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("abhishek/autonlp-japanese-sentiment-59363", num_labels=2) # You can load base model from huggingface

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00


In [ ]:
# Define the evaluation metrics
import numpy as np
import evaluate

# Load the metrics
load_accuracy = evaluate.load("accuracy")
load_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}


In [ ]:
# Log in to your Hugging Face account
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_path = '/content/drive/MyDrive/Fine Tuned model/text_classification/japanese_model'

In [ ]:
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=model_path,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss
500,0.027400
1000,0.014200
1500,0.001000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation 

TrainOutput(global_step=1570, training_loss=0.01357012914499612, metrics={'train_runtime': 981.6668, 'train_samples_per_second': 25.467, 'train_steps_per_second': 1.599, 'total_flos': 1541894506618320.0, 'train_loss': 0.01357012914499612, 'epoch': 10.0})

In [ ]:
# Compute the evaluation metrics
trainer.evaluate()

{'eval_loss': 1.9986831830465235e-05,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_runtime': 2.0254,
 'eval_samples_per_second': 154.045,
 'eval_steps_per_second': 9.875,
 'epoch': 10.0}

In [ ]:
trainer.save_model(model_path)

# 4. Analyzing new data with the model

In [ ]:
# Upload the model to the Hub
# trainer.push_to_hub()

In [ ]:
from transformers import pipeline

# Specify the task and model directory
sentiment_model = pipeline(task="sentiment-analysis", model=model_path)

# Run inferences with your model
sentiment_model(["愛してる", "私はあなたを殺します", "あなたは時間の無駄です", "今日は私のリーグの最高の日です", "おいしい！砂糖をまぶした赤い果実と、モリッシュ、シナモン、アロマの深みのある甘さ。"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'positive', 'score': 0.9987404942512512},
 {'label': 'negative', 'score': 0.9920315742492676},
 {'label': 'negative', 'score': 0.9983945488929749},
 {'label': 'positive', 'score': 0.9997242093086243},
 {'label': 'positive', 'score': 0.9997072815895081}]